In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import openTSNE

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torchvision

import shapely

import os
import sys

sys.path.append("../scripts")
from data import SmallPatchesDataset, get_filenames
from models import AutoEncoder, DEC

# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
n_cities_used = 100

# remove patches that are have small intersection with block
intersection_threshold = 0.25

# remove patches from blocks that have more than a threshold 
patches_count_max = 50

In [3]:
blocks_df = gpd.read_file("../data/census_blocks_patches_v2.geojson")

# filtering to test with some blocks, selecting the cities with the biggest number of blocks
biggest_blocks = blocks_df.groupby(["state", "county"]).agg("count").reset_index().iloc[:, 0:3].sort_values("tract", ascending = False).head(n_cities_used)
blocks_df = blocks_df[(blocks_df.state.isin(biggest_blocks.state) & blocks_df.county.isin(biggest_blocks.county))]

# cleaning blocks with missing data
blocks_df = blocks_df[blocks_df.mhi > 0]
blocks_df = blocks_df.dropna()
blocks_df = blocks_df[blocks_df.patches_relation.apply(len) > 0]

blocks_df["area_km2"] = blocks_df['geometry'].to_crs({'proj':'cea'}).area / 10**6
blocks_df["density"] = blocks_df["pop"] / blocks_df["area_km2"]

In [4]:
def clean_patches_relation(s):
    s = s.split("\n")
    s = dict([x.split(":") for x in s])
    filenames = []
    data = []
    for key, value in s.items():
        value = value.split(" ")
        idx = np.array([float(v) for v in value[0].split(",")])
        ratio = np.array([float(v) for v in value[1].split(",")])
        idx = idx[ratio > intersection_threshold]
        ratio = ratio[ratio > intersection_threshold]
        for i in range(len(idx)):
            data.append([idx[i], ratio[i]])
            filenames.append(key)
    data = np.array(data)
    if len(filenames) > patches_count_max:
        selected = np.random.choice(
            len(filenames),
            size=patches_count_max,
            replace=False,
            p=data[:, 1] / data[:, 1].sum(),
        )
        data = data[selected, :]
        filenames = [filenames[i] for i in selected]
    return [filenames, data]

blocks_df["clean_patches_relation"] = blocks_df.patches_relation.apply(
    clean_patches_relation
)
blocks_df["n_patches"] = blocks_df["clean_patches_relation"].apply(
    lambda x: x[1].shape[0]
)
blocks_df = blocks_df[blocks_df.n_patches > 0]

In [5]:
blocks_df = blocks_df.reset_index(drop = True)

In [6]:
patches_blocks = {}
for i, row in blocks_df.iterrows():
    relation_list = row["patches_relation"].strip(" ").split(" ")
    relation_list = [x.split(";") for x in relation_list]
    relation_list = row["clean_patches_relation"][0]
    idx = row["clean_patches_relation"][1][:, 0]
    files = [f"{relation_list[j]} {int(idx[j])}" for j in range(len(relation_list))]
    for file in files:
        if file in patches_blocks.keys():
            patches_blocks[file].append(i)
        else:
            patches_blocks[file] = [i]
print(f"Number of unique patches: {len(patches_blocks.keys())}")

Number of unique patches: 2214833


In [7]:
model_resnet50 = torchvision.models.resnet50(weights = "DEFAULT")
model_resnet50 = nn.Sequential(*(list(model_resnet50.children())[:-1] + [nn.Flatten()]))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet50 = model_resnet50.to(device)

filenames = list(patches_blocks.keys())
filenames = np.random.choice(filenames, size = 200000, replace = False)
dataset = SmallPatchesDataset(filenames, resize = (224, 224))
dl = torch.utils.data.DataLoader(dataset, batch_size = 300)

In [8]:
embeddings = []
with torch.no_grad():
    for x in tqdm(dl):
        x = x.to(device)
        embeddings.append(model_resnet50(x).squeeze().cpu().numpy())
embeddings = np.concatenate(embeddings, axis = 0)

  0%|          | 0/667 [00:00<?, ?it/s]

/home/giovani/miniconda3/envs/urban_gdp/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 667/667 [14:49<00:00,  1.33s/it]


In [9]:
embeddings_proj = openTSNE.TSNE(
    initialization="pca",
    perplexity = 250,
    #exaggeration = 4,
    metric = "cosine",
    n_jobs=32
).fit(embeddings[:50000, :])
embeddings_proj = embeddings_proj.prepare_partial(embeddings)
kmeans = KMeans(n_clusters = 30, random_state = 0).fit(embeddings_proj)

/home/giovani/miniconda3/envs/urban_gdp/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [10]:

k = 30
latent_dim = 2048
cluster_centers_idx = [np.argmin(np.linalg.norm(embeddings_proj - kmeans.cluster_centers_[i], axis=1))
                       for i in range(k)]
centroids = torch.tensor(embeddings[cluster_centers_idx, :]).to(device)

model_dec = DEC(
    n_clusters = k,
    embedding_dim = 2048,
    encoder = model_resnet50,
    cluster_centers = centroids,
)
model_dec = model_dec.to(device)

In [11]:
filenames = list(patches_blocks.keys())
dataset = SmallPatchesDataset(filenames, resize = (224, 224))
dl = torch.utils.data.DataLoader(dataset, batch_size = 300)

In [12]:
def get_clusters(dl, model):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    clusters = []
    clusters_distance = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dl):
            batch = batch.to(device)
            c = model(batch).detach().cpu().numpy()
            d = model.centroids_distance(batch).detach().cpu().numpy()
            clusters.append(c)
            clusters_distance.append(d)

    clusters = np.concatenate(clusters).argmax(axis = 1)
    clusters_distance = np.concatenate(clusters_distance)
    return clusters, clusters_distance


In [13]:
clusters, clusters_distance = get_clusters(dl, model_dec)

  0%|          | 0/7383 [00:00<?, ?it/s]

/home/giovani/miniconda3/envs/urban_gdp/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 7383/7383 [2:50:29<00:00,  1.39s/it]  


## Comparison of all methods

In [14]:
# functions to generate features from model output
def count_of_patches_cluster():
    x = np.zeros((blocks_df.shape[0], k))
    for i, (file, cluster) in enumerate(zip(filenames, clusters)):
        for b in patches_blocks[file]:
            x[b, cluster] += 1
    x = pd.DataFrame(x, columns = [f"cluster_{i}" for i in range(k)])
    x = x.loc[:, x.sum(axis = 0) > 0]
    return x

def fraction_of_patches_cluster():
    x = np.zeros((blocks_df.shape[0], k))
    for i, (file, cluster) in enumerate(zip(filenames, clusters)):
        for b in patches_blocks[file]:
            x[b, cluster] += 1
    x_sum = x.sum(axis = 1)
    x = x / x_sum[:, None]
    x = pd.DataFrame(x, columns = [f"cluster_{i}" for i in range(k)])
    x = x.loc[:, x.sum(axis = 0) > 0]
    x["count"] = x_sum
    return x

def distance_of_patches_cluster():
    x = np.zeros((blocks_df.shape[0], k))
    for i, (file, distances) in enumerate(zip(filenames, clusters_distance)):
        for b in patches_blocks[file]:
            x[b] += distances
    x_sum = blocks_df.n_patches.values.reshape(-1)
    x = x / x_sum[:, None]
    x = pd.DataFrame(x, columns = [f"cluster_{i}" for i in range(k)])
    x = x.loc[:, x.sum(axis = 0) > 0]
    x["count"] = x_sum
    return x

In [15]:
def get_x(method):
    if method == "count":
        return count_of_patches_cluster()
    elif method == "fraction":
        return fraction_of_patches_cluster()
    elif method == "distance":
        return distance_of_patches_cluster()

In [16]:
def eval(clf, x_train, y_train, x_test, y_test):
    mae_train = mean_absolute_error(y_train, clf.predict(x_train))
    r2_train = r2_score(y_train, clf.predict(x_train))
    
    mae_test = mean_absolute_error(y_test, clf.predict(x_test))
    r2_test = r2_score(y_test, clf.predict(x_test))
    return r2_train, r2_test

In [17]:
def grid_search_rf(x_train, y_train, x_test, y_test):
    rf = RandomForestRegressor()
    parameters = {
        "n_estimators": [10, 100, 1000],
        "max_depth": [10, 100],
        #"min_samples_split": [2, 10, 100],
    }
    clf = GridSearchCV(rf, parameters, n_jobs=-1)
    clf.fit(x_train, y_train)
    return eval(clf, x_train, y_train, x_test, y_test)

In [18]:
class MLP(nn.Module):
    def __init__(self, dims):
        super(MLP, self).__init__()
        self.layers = []
        for in_dim, out_dim in zip(dims[:-1], dims[1:]):
            self.layers.append(nn.Linear(in_dim, out_dim))
            if out_dim != dims[-1]:
                self.layers.append(nn.ReLU())
        self.layers = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.layers(x)

    def predict(self, x):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.eval()
        if type(x) == pd.DataFrame:
            x_ = torch.from_numpy(x.values)
        elif type(x) == np.ndarray:
            x_ = torch.from_numpy(x)
            
        with torch.no_grad():
            x_ = x_.to(device)
            y = self.layers(x_)
            return y.detach().cpu().numpy()


In [19]:

def train_mlp(model, dl_train, dl_test):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    test_loss = []
    for i in range(100):
        iter_loss = 0
        for x, y in dl_train:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            iter_loss += loss.item()

        if i % 3 == 0:
            iter_loss = 0
            with torch.no_grad():
                for x, y in dl_test:
                    x, y = x.to(device), y.to(device)
                    y_pred = model(x)
                    loss = criterion(y_pred, y)
                    iter_loss += loss.item()
                test_loss.append(iter_loss)

            if i > 10 and test_loss[-1] > test_loss[-2]:
                break        

def grid_search_mlp(x_train, y_train, x_test, y_test):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    idx_train_, idx_val = train_test_split(np.arange(x_train.shape[0]), test_size = 0.2, random_state = 0)
    x_val_, y_val_ = x_train.values[idx_val, :], y_train[idx_val]
    x_train_, y_train_ = x_train.values[idx_train_, :], y_train[idx_train_]
    x_test_, y_test_ = x_test.values, y_test
    scaler = StandardScaler()
    x_train_ = scaler.fit_transform(x_train_)
    x_val_ = scaler.transform(x_val_)
    x_test_ = scaler.transform(x_test_)
    dl_train_ = DataLoader(TensorDataset(torch.tensor(x_train_), torch.tensor(y_train_.reshape(-1, 1))), batch_size = 128)
    dl_val = DataLoader(TensorDataset(torch.tensor(x_val_), torch.tensor(y_val_).reshape(-1, 1)), batch_size = 128)

    best_r2 = -np.inf
    best_model = None
    for dims in [[x_train.shape[1], 32, 64, 32, 1], [x_train.shape[1], 64, 256, 32, 1], [x_train.shape[1], 64, 512, 128, 1]]:
        model_1 = MLP(dims)
        model_1.to(device, dtype = torch.double)
        train_mlp(model_1, dl_train_, dl_val)
        r2_train, r2_test = eval(model_1, x_train_, y_train_, x_val_, y_val_)

        if r2_test > best_r2:
            best_r2 = r2_test
            best_model = model_1
    
    return eval(model_1, x_train_, y_train_, x_test_, y_test_)

In [20]:
idx_train, idx_test = train_test_split(np.arange(blocks_df.shape[0]), test_size = 0.2, random_state = 0)
df_results = []
for method in ["count", "fraction", "distance"]:
    x = get_x(method)
    x_train, x_test = x.loc[idx_train, :], x.loc[idx_test, :]
    for target in ["mhi", "ed_attain", "pop"]:
        y_train, y_test = blocks_df[target].values[idx_train], blocks_df[target].values[idx_test] 
        
        for regression in ["mlp", "rf"]:
            if regression == "rf":
                r2_train, r2_test = grid_search_rf(x_train, y_train, x_test, y_test)
            elif regression == "mlp":
                r2_train, r2_test = grid_search_mlp(x_train, y_train, x_test, y_test)

            print(f"method: {method}, target: {target}, regression: {regression}, r2_train: {r2_train:.2f}, r2_test: {r2_test:.2f}")
            df_results.append([method, target, regression, r2_train, r2_test])

method: count, target: mhi, regression: mlp, r2_train: 0.08, r2_test: 0.07
method: count, target: mhi, regression: rf, r2_train: 0.16, r2_test: 0.06
method: count, target: ed_attain, regression: mlp, r2_train: 0.11, r2_test: 0.06
method: count, target: ed_attain, regression: rf, r2_train: 0.15, r2_test: 0.07
method: count, target: pop, regression: mlp, r2_train: 0.08, r2_test: 0.06
method: count, target: pop, regression: rf, r2_train: 0.17, r2_test: 0.06
method: fraction, target: mhi, regression: mlp, r2_train: 0.08, r2_test: 0.07
method: fraction, target: mhi, regression: rf, r2_train: 0.16, r2_test: 0.06
method: fraction, target: ed_attain, regression: mlp, r2_train: 0.11, r2_test: 0.06
method: fraction, target: ed_attain, regression: rf, r2_train: 0.15, r2_test: 0.06
method: fraction, target: pop, regression: mlp, r2_train: 0.08, r2_test: 0.07
method: fraction, target: pop, regression: rf, r2_train: 0.18, r2_test: 0.08
method: distance, target: mhi, regression: mlp, r2_train: 0.02, 

In [24]:
pd.DataFrame(df_results, columns = ["method", "target", "regression", "r2_train", "r2_test"]
             ).sort_values("r2_test", ascending = False)

,method,target,regression,r2_train,r2_test
15,distance,ed_attain,rf,0.887753,0.170019
13,distance,mhi,rf,0.882295,0.140973
17,distance,pop,rf,0.233911,0.094468
14,distance,ed_attain,mlp,0.078200,0.082790
11,fraction,pop,rf,0.181579,0.076680
10,fraction,pop,mlp,0.082474,0.069305
3,count,ed_attain,rf,0.152897,0.065851
6,fraction,mhi,mlp,0.075914,0.065801
0,count,mhi,mlp,0.077167,0.065621
9,fraction,ed_attain,rf,0.150939,0.064532
